In [1]:
!pip install -qU kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [3]:
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download ryanrudes/yttts-speech

100% 9.90G/9.91G [01:26<00:00, 190MB/s]
100% 9.91G/9.91G [01:26<00:00, 124MB/s]


In [5]:
!unzip yttts-speech.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/audio.wav  
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/audio.wav  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/audio.wav  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/audio.wav  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/audio.wav  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/audio.wav  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:41,780-00:03:45,660/audio.wav  
  inflating: data/yexR53My2O4/00:03:41,78

In [6]:
!pip install -qU bs4 tqdm datasets

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.3 MB/s eta 0:00:00


In [7]:
import os
import time
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [8]:
video_ids = os.listdir("data")
video_ids[:5]

['YBlNQK0Ao6g', 'YqvhDPd1UEw', 'NTz4rJS9BAI', 'P0yVuoATjzs', '7Q2JhZxNPow']

In [9]:
splits = sorted(os.listdir(f"data/{video_ids[0]}"))
splits[:5]

['00:00:00,199-00:00:02,879',
 '00:00:02,879-00:00:05,520',
 '00:00:05,520-00:00:08,970',
 '00:00:08,970-00:00:12,509',
 '00:00:12,509-00:00:15,780']

In [10]:
with open(f"data/{video_ids[0]}/{splits[0]}/subtitles.txt") as f:
    text = f.read()
print(text)

okay I'm sure many of you have already


In [11]:
documents = []
for video_id in tqdm(video_ids):
    splits = sorted(os.listdir(f"data/{video_id}"))
    # we start at 00:00:00
    start_timestamp = "00:00:00"
    passage = ""
    for i, s in enumerate(splits):
        with open(f"data/{video_id}/{s}/subtitles.txt") as f:
            # append tect to current chunk
            out = f.read()
            passage += " " + out
        # average sentence length is 75-100 characters so we will cut off
        # around 3-4 sentences
        if len(passage) > 360:
            # now we've hit the needed length create a record
            # extract the end timestamp from the filename
            end_timestamp = s.split("-")[1].split(",")[0]
            # extract string timestamps to actual datetime objects
            start = time.strptime(start_timestamp,"%H:%M:%S")
            end = time.strptime(end_timestamp,"%H:%M:%S")
            # now we extract the second/minute/hour values and convert
            # to total number of seconds
            start_second = start.tm_sec + start.tm_min*60 + start.tm_hour*3600
            end_second = end.tm_sec + end.tm_min*60 + end.tm_hour*3600
            # save this to the documents list
            documents.append({
                "video_id": video_id,
                "text": passage,
                "start_second": start_second,
                "end_second": end_second,
                "url": f"https://www.youtube.com/watch?v={video_id}&t={start_second}s",
            })
            # now we update the start_timestamp for the next chunk
            start_timestamp = end_timestamp
            # refresh passage
            passage = ""

  0%|          | 0/127 [00:00<?, ?it/s]

In [12]:
documents[:3]

[{'video_id': 'YBlNQK0Ao6g',
  'text': " okay I'm sure many of you have already seen this because it was rather widely announced but the open AI team has announced a new model that produces pictures instead of text so as you can see right here on the Left you'll always see like a half a picture and on the right is the ground truth so they took this picture they simply cut the bottom half right here and then they let the model sort of",
  'start_second': 0,
  'end_second': 30,
  'url': 'https://www.youtube.com/watch?v=YBlNQK0Ao6g&t=0s'},
 {'video_id': 'YBlNQK0Ao6g',
  'text': " imagine what they cut away and what it comes up with is pretty cool I have to say like look at the birds like this is just awesome but the special thing about this isn't that it simply completes pictures the special thing about it is it does it one pixel by pixel so basically it goes at this pixel right here and asks ok what's that pixel and then what's that pixel and then what's",
  'start_second': 30,
  'end_se

In [13]:
import lxml  # if on mac, pip/conda install lxml

metadata = {}
for _id in tqdm(video_ids):
    r = requests.get(f"https://www.youtube.com/watch?v={_id}")
    soup = BeautifulSoup(r.content, 'lxml')  # lxml package is used here
    try:
        title = soup.find("meta", property="og:title").get("content")
        thumbnail = soup.find("meta", property="og:image").get("content")
        metadata[_id] = {"title": title, "thumbnail": thumbnail}
    except Exception as e:
        print(e)
        print(_id)
        metadata[_id] = {"title": "", "thumbnail": ""}

len(metadata)

  0%|          | 0/127 [00:00<?, ?it/s]

'NoneType' object has no attribute 'get'
Y6RRHw9uN9o
'NoneType' object has no attribute 'get'
bM9BXu9lPZ0
'NoneType' object has no attribute 'get'
xtOg44r6dsE
'NoneType' object has no attribute 'get'
arbbhHyRP90
'NoneType' object has no attribute 'get'
fpDaQxG5w4o


127

In [14]:
metadata['ZPewmEu7644']

{'title': 'GANS for Semi-Supervised Learning in Keras (7.4)',
 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}

In [15]:
for i, doc in enumerate(documents):
    _id = doc['video_id']
    meta = metadata[_id]
    # add metadata to existing doc
    documents[i] = {**doc, **meta}

In [16]:
documents[0]

{'video_id': 'YBlNQK0Ao6g',
 'text': " okay I'm sure many of you have already seen this because it was rather widely announced but the open AI team has announced a new model that produces pictures instead of text so as you can see right here on the Left you'll always see like a half a picture and on the right is the ground truth so they took this picture they simply cut the bottom half right here and then they let the model sort of",
 'start_second': 0,
 'end_second': 30,
 'url': 'https://www.youtube.com/watch?v=YBlNQK0Ao6g&t=0s',
 'title': 'Image GPT: Generative Pretraining from Pixels (Paper Explained)',
 'thumbnail': 'https://i.ytimg.com/vi/YBlNQK0Ao6g/maxresdefault.jpg'}

In [27]:
documents[0]['text']

" okay I'm sure many of you have already seen this because it was rather widely announced but the open AI team has announced a new model that produces pictures instead of text so as you can see right here on the Left you'll always see like a half a picture and on the right is the ground truth so they took this picture they simply cut the bottom half right here and then they let the model sort of"

In [66]:
import pickle
with open("train.pkl", "wb") as f:
    pickle.dump(documents, f)

In [67]:
with open("train.pkl", "rb") as f:
    data = pickle.load(f)

{'video_id': 'YBlNQK0Ao6g',
 'text': " okay I'm sure many of you have already seen this because it was rather widely announced but the open AI team has announced a new model that produces pictures instead of text so as you can see right here on the Left you'll always see like a half a picture and on the right is the ground truth so they took this picture they simply cut the bottom half right here and then they let the model sort of",
 'start_second': 0,
 'end_second': 30,
 'url': 'https://www.youtube.com/watch?v=YBlNQK0Ao6g&t=0s',
 'title': 'Image GPT: Generative Pretraining from Pixels (Paper Explained)',
 'thumbnail': 'https://i.ytimg.com/vi/YBlNQK0Ao6g/maxresdefault.jpg'}

In [72]:
text = []
for i in data:
  text.append(i['text'])


In [74]:
text[0]

" okay I'm sure many of you have already seen this because it was rather widely announced but the open AI team has announced a new model that produces pictures instead of text so as you can see right here on the Left you'll always see like a half a picture and on the right is the ground truth so they took this picture they simply cut the bottom half right here and then they let the model sort of"

In [75]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.0 MB/s eta 0:00:00


In [56]:
# import json

# with open("train.jsonl", "w") as f:
#     for doc in documents:
#         json.dump(doc, f)
#         f.write('\n')

In [57]:
# with open("train.jsonl") as f:
#     d = f.readlines()

In [76]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=37af40a789c3012d7f4977609d4f975dd803e292921b6d966f00d9f66a604f78
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [77]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(text)
sentence_embeddings.shape

(11298, 768)

In [78]:
sentence_embeddings[0]

array([-7.54201189e-02,  1.92189589e-02,  1.96473464e-01,  2.67315865e-01,
       -2.98080951e-01, -1.15373373e+00,  7.27740824e-02,  3.95041376e-01,
        3.64883810e-01, -2.93056428e-01, -9.76201817e-02,  7.80188978e-01,
        2.74647206e-01,  1.54961869e-01, -7.46797323e-01,  3.89144540e-01,
       -7.27310404e-02,  1.50384858e-01, -2.48145387e-01, -3.24022681e-01,
       -2.66419530e-01, -7.91859865e-01, -6.15505651e-02,  5.17503954e-02,
        4.96042311e-01,  1.75048620e-01,  1.42479241e-01, -4.04419392e-01,
        1.37473226e-01,  8.52926791e-01, -3.96374613e-01, -1.61805585e-01,
       -7.82142058e-02, -2.33709604e-01, -3.74882638e-01,  4.02803898e-01,
        7.62203097e-01,  4.35533941e-01,  5.20134628e-01,  3.18067968e-01,
       -2.89005786e-01, -6.84408285e-03, -4.04285900e-02, -7.16074347e-01,
       -9.70781207e-01, -5.03963888e-01,  1.04116812e-01,  8.02670479e-01,
        6.07910633e-01, -9.33364570e-01,  7.04476833e-02,  6.67523965e-02,
        2.47721687e-01,  

In [96]:
import faiss
index = faiss.IndexFlatL2(768)

In [97]:
index.add(sentence_embeddings)


In [98]:
index.ntotal

11298

In [109]:
k = 10
xq = model.encode(["Github how to upload a file more than 25 mb"])


In [110]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 295 2788 5109 5992 5529 3515 4924 2787 7499  458]]
CPU times: user 14.3 ms, sys: 0 ns, total: 14.3 ms
Wall time: 16.1 ms


In [111]:
st = I[0]

for i in st:
  print(documents[i])

{'video_id': 'YqvhDPd1UEw', 'text': " rainbow DQ and simple and rainbow DQ and curl seen every out performs prior and state-of-the-art and it's getting at about 25 percent of human normalized score here is a broken out for the individual games and curls outperforming proxy they are fairly consistently what's simple coming in first on it's still two of them so computers are all matched human data", 'start_second': 5847, 'end_second': 5873, 'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=5847s', 'title': 'L12 Representation Learning for Reinforcement Learning --- CS294-158 UC Berkeley Spring 2020', 'thumbnail': 'https://i.ytimg.com/vi/YqvhDPd1UEw/maxresdefault.jpg'}
{'video_id': '-Y7PLaxXUrs', 'text': " And to see how much this thing is now permitting the modern world, just note that almost 30 percent of the awesome computational power for inference and all these Google Data Centers, all these data centers of Google, all over the world, is used for LSTM. Almost 30 percent. If you h

In [113]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(768)
index2 = faiss.IndexIVFFlat(quantizer, 768, nlist)

In [114]:
index2.is_trained

False

In [115]:
index2.train(sentence_embeddings)
index2.is_trained

True

In [117]:
index2.add(sentence_embeddings)
index2.ntotal

11298

In [122]:
%%time
D, I = index2.search(xq, k)  # search
print(I[0])

[  295  2788  5109  5529  3515  2787   458  7993 10296  5522]
CPU times: user 206 µs, sys: 1.01 ms, total: 1.22 ms
Wall time: 776 µs


In [123]:
st = I[0]

for i in st:
  print(documents[i])

{'video_id': 'YqvhDPd1UEw', 'text': " rainbow DQ and simple and rainbow DQ and curl seen every out performs prior and state-of-the-art and it's getting at about 25 percent of human normalized score here is a broken out for the individual games and curls outperforming proxy they are fairly consistently what's simple coming in first on it's still two of them so computers are all matched human data", 'start_second': 5847, 'end_second': 5873, 'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=5847s', 'title': 'L12 Representation Learning for Reinforcement Learning --- CS294-158 UC Berkeley Spring 2020', 'thumbnail': 'https://i.ytimg.com/vi/YqvhDPd1UEw/maxresdefault.jpg'}
{'video_id': '-Y7PLaxXUrs', 'text': " And to see how much this thing is now permitting the modern world, just note that almost 30 percent of the awesome computational power for inference and all these Google Data Centers, all these data centers of Google, all over the world, is used for LSTM. Almost 30 percent. If you h

In [ ]:
[[ 295 2788 5109 5992 5529 3515 4924 2787 7499  458]]
 [  295  2788  5109  5529  3515  2787   458  7993 10296  5522]


In [126]:
%%time
index2.probe = 10

D, I = index2.search(xq, k)  # search
print(I[0])

[  295  2788  5109  5529  3515  2787   458  7993 10296  5522]
CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 2.68 ms


In [ ]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=0a4e64126561968bdbe965f1b5d53bca033cda225af0840efa0b08e429e7e9f2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-trans

In [ ]:
from datasets import load_dataset

yt = load_dataset(
    'pinecone/yt-transcriptions',
    split = 'train',
    revision = '926a45'

)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-c0a57d2f8951521b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [ ]:
for x in yt:
  print(x)
  break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
embed_dim = retriever.get_sentence_embedding_dimension()

In [ ]:
embed_dim

768

In [ ]:
import pinecone

pinecone.init(
    api_key="05d04c60-f94b-4e56-9d98-3dcbe5cbf914",
    environment="us-west4-gcp-free"
)

# create index
pinecone.create_index(
    "youtube-search",
    dimension=embed_dim,
    metric="cosine"
)

# connect to new index
index = pinecone.Index("youtube-search")

In [ ]:
from tqdm.auto import tqdm
docs = []
batch_size = 64

for i in tqdm(range(0,len(yt),batch_size)):

  i_end = min(i+batch_size,len(yt))
  batch = yt[i:i_end]                          # extract batch from YT transactions
  embeds = retriever.encode(batch['text']).tolist()   # encode a batch
  ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]

  meta = [{
      'video_id': x[0],
      'title': x[1],
      'text': x[2],
      'start_second': x[3],
      'end_second': x[4],
      'url': x[5],
      'thumbnail': x[6]
  } for x in zip(
      batch['video_id'],
      batch['title'],
      batch['text'],
      batch['start_second'],
      batch['end_second'],
      batch['url'],
      batch['thumbnail']
  )]

  to_upsert = list(zip(ids,embeds,meta))   # create list of (IDs, vectors, metadata) to upsert
  index.upsert(vectors = to_upsert)        # adding to pinecone

  0%|          | 0/177 [00:00<?, ?it/s]

In [ ]:
index.describe_index_stats

<bound method Index.describe_index_stats of <pinecone.index.Index object at 0x7fda702548b0>>

# Querying

In [ ]:
query = 'what is principal component analysis?'
xq = retriever.encode([query]).tolist()

In [ ]:
xc = index.query(xq, top_k = 3, include_metadata = True)
for context in xc['matches']:
  print(context['metadata']['text'], end = '\n')
  print('url',context['metadata']['url'])

 is something red concepts in the same places so this is a PCA and then blue I think these blue is the attention codes for identify the red things and in red or the generation code for make something red and they will be put in the same place which is pretty cool it means that the energy function really learns the feature of something being red I find this pretty pretty neat and then
url https://www.youtube.com/watch?v=Cs_j-oNwGgg&t=1933s
 in PCA the the the region of the space that is perfectly reconstructed is the principal subspace right in this case here if the data points are sampled from this from the sparrow the principal subspace here dimension 1 is this so this has become our reconstruction error 0 and everything else grows could racket quadratically right because you take a point you project it on this and so if
url https://www.youtube.com/watch?v=A7AnCvYDQrU&t=1922s
 think of CPC is trying to do something like multi-step prediction tasks so if you look at these multiple pred

In [ ]:
xc

{'matches': [{'id': 'NTz4rJS9BAI-140',
              'metadata': {'end_second': 158.0,
                           'start_second': 140.0,
                           'text': ' upset some of my colleagues nobly Mike '
                                   'Jordan but I assess on my college might '
                                   "say this is all machine learning is it's "
                                   'just predict making predictions from '
                                   'examples I have a set of something I have '
                                   'a set of something else and I want to find '
                                   'prediction functions that map X to Y I '
                                   "know there's more than that but this is "
                                   'the backbone this is why all of you are '
                                   "here this is why they're",
                           'thumbnail': 'https://i.ytimg.com/vi/NTz4rJS9BAI/maxresdefault.jpg',
